# Portfolio Sorts

This notebook implements a simple portfolio sort.

## Load Packages and Extra Functions

In [1]:
using Dates, DelimitedFiles, Statistics

include("jlFiles/printmat.jl")

printTable2

## Load Data

The data set contains daily data for "dates", the equity market return, riskfree rate and the the returns of the 25 Fama-French portfolios. All returns are in percent.

In [2]:
x   = readdlm("Data/MomentumSR.csv",',')
dN  = Date.(x[:,1],"yyyy-mm-dd")                  #Julia dates 
y   = convert.(Float64,x[:,2:end])

(Rm,Rf,R) = (y[:,1],y[:,2],y[:,3:end])

println("\nThe first few rows of dN, Rm and Rf")
printmat([dN[1:4] Rm[1:4] Rf[1:4]])

println("size of dN, Rm, Rf, R")
println(size(dN),"\n",size(Rm),"\n",size(Rf),"\n",size(R))

(T,n) = size(R);                      #number of periods and assets


The first few rows of dN, Rm and Rf
1979-01-02     0.615     0.035
1979-01-03     1.155     0.035
1979-01-04     0.975     0.035
1979-01-05     0.685     0.035

size of dN, Rm, Rf, R
(9837,)
(9837,)
(9837,)
(9837, 25)


## Implementing the Portfolio Sorts

1.  Sort  $R_{t-1}$  across the 25 assets.
2. (In the evening of) period  $t-1$, construct the `Hi` portfolio by buying 1/5 of each of the 5 best assets based on the sort in point 1. Similarly, construct the `Lo` portfolio by buying 1/5 of each of the 5 worst assets. Collect these portfolio weights in the vectors `wHi` and `wLo`.
3. In period  $t$ , the return on the `Hi` portfolio is $R_{p,t}=wHi'R_{t}$. Similarly for `Lo`.
4. Repeat for all periods

In [3]:
(RHi,RLo) = (fill(NaN,T),fill(NaN,T))
for t = 2:T         #loop over periods, save portfolio returns
    #local sort1, wHi, wLo          #only needed in REPL/script
    sort1                  = sortperm(R[t-1,:])  #sort1[1] is index of worst asset
    (wLo,wHi)              = (zeros(n),zeros(n))
    wLo[sort1[1:5]]       .= 1/5    #equally weighted inside Lo portfolio
    wHi[sort1[end-4:end]] .= 1/5
    RLo[t]                 = wLo'R[t,:]
    RHi[t]                 = wHi'R[t,:]
end

ReLo = RLo[2:end] - Rf[2:end]
ReHi = RHi[2:end] - Rf[2:end];       #cut out t=1, excess returns

Calculate the mean (excess) return, its standard deviation and the Sharpe ratio. Annualize by assuming 252 trading days per year. Compare with the excess return on passively holding an equity market index.

In [4]:
```

Calculate average excess return, the std and the SR ratio - and annualise all of them.
```
function ReturnStats(Re,Annfactor=252)
    μ  = mean(Re)*Annfactor
    σ  = std(Re)*sqrt(Annfactor)
    SR = μ/σ
    stats = [μ,σ,SR]
    return stats
end

ReturnStats (generic function with 2 methods)

In [5]:
Rme = Rm - Rf

StatLo = ReturnStats(ReLo,252)
StatHi = ReturnStats(ReHi,252)
Statm  = ReturnStats(Rme[2:end],252)

printblue("Stats for the portfolio returns, annualized:")
printTable([StatLo StatHi Statm],["Lo" "Hi" "market"],["μ";"σ";"SR"])

Stats for the portfolio returns, annualized:
          Lo        Hi    market
μ      1.651    17.914     8.374
σ     18.706    17.610    16.837
SR     0.088     1.017     0.497

